In [14]:
# PRIME GENERATOR

from random import randrange, getrandbits
def is_prime(n, k = 64):
    if n == 2:
        return True
    elif n == 1:
        return False
    elif n % 2==0 and n != 2:
        return False
    
    #n - 1 = r*2^s find r and s
    r = n - 1
    s = 0
    while r % 2 == 0:
        r //= 2
        s+= 1

    for i in range(k):
        if miller_rabin(r, n, s) == False:
            return False
    else:
        return True
    

def miller_rabin(r, n, s):
     #find r, s (n-1) = r*2^s

    j = 2
    a = randrange(1, n-1)
    fermat_test =  pow(a, r, n)
    if fermat_test == 1 or fermat_test == n - 1:
        return True
    while j < s:

        x = pow(a, r*pow(2, j), n)
        if x == (1%n):
            return True
        elif x== -1:
            return False
        j += 1
    else:
        x = pow(a, r*pow(2, s-1), n)
        
        euler_test = pow(a, int((n-1)/2), n)
       
        if x == euler_test and abs(x) == 1:
            return True
        else:
            return False
        
        
        
def generate_candidate(length = 32):
    p = getrandbits(length)
    p = p | (1 << length - 1) | 1
    return p


def generate_prime(length = 32):
    result = False
    while result == False:
        candidate = generate_candidate(length)
        result = is_prime(candidate)
    return candidate 

In [20]:
# DEFs

import random as rd

def extended_gcd(a, b):
    '''Extended Euclidean Algorithm to find gcd and coefficients'''
    if b == 0:
        return a, 1, 0

    gcd, x1, y1 = extended_gcd(b, a % b)
    x = y1
    y = x1 - (a // b) * y1
    return gcd, x, y

def gcd(p, q):
    '''returns the greatest common factor of two numbers'''
    return extended_gcd(p,q)[0]

def phi_gen(p, q):
    '''returns the totient of two PRIME numbers'''
    return (p - 1) * (q - 1)

def n_gen(p, q):
    '''returns n'''
    return p * q

def e_gen(p, q):
    '''returns e'''
    tot = phi_gen(p, q)
    for e in range(17, tot):
        if gcd(e, tot) == 1:
            break
    return e

def d_gen(p, q):
    '''returns d'''
    gcd, x, _ = extended_gcd(e_gen(p, q), phi_gen(p, q))
    return x % phi_gen(p, q)

def key_gen():
    p, q = generate_prime(1024), generate_prime(1024) # the parentheses can be left blank, but have 512 will make the prime much larger
    n = n_gen(p, q)
    e = e_gen(p, q)
    d = d_gen(p, q)
    phi = phi_gen(p, q)
    return (n, e, d)


def convert(string):
    """string --> zero-padded ASCII (as a str. as an int would cause issue: it will kill the trailing zeros)"""
    code = ''
    for char in string:
        code += f"{ord(char):03}"  # force 3 digits per char
    return code

def convert_back(decrypted_code_str):
    """zero-padded ASCII string --> original string"""
    message = ""
    # Read in chunks of 3
    for i in range(0, len(decrypted_code_str), 3):
        message += chr(int(decrypted_code_str[i:i+3]))
    return message

def encryption(message, n, e):
    '''C = m^e mod n'''
    """Return (ciphertext, length_of_ascii_str)."""
    code_str = convert(message)          
    m_len = len(code_str) # measure how many digits in total
    m_int = int(code_str) # convert to int for RSA (will result in loss of trailing zero)
    c = pow(m_int, e, n)  # encrypt
    return c, m_len


def decryption(cipher_pair, n, d):
    """Given (ciphertext, length), return the original message string."""
    c, m_len = cipher_pair
    decrypted_int = pow(c, d, n)  # standard RSA decryption
    decrypted_str = str(decrypted_int).rjust(m_len, '0') # rjust put the right amount of trailing zeros so that the str reaches an desired length (m_len)
    return convert_back(decrypted_str)

In [22]:
# GEN KEY

n, e, d = key_gen()
print("Public Key: (n, e)", (n, e))
print("Private Key: (n, d)", (n, d))

Public Key: (n, e) (23056235814536983572965195495200210211247183571757472039868723634192761256517774287961785737973733429885640339130246268450034644258438795728841338363402718787882084497854650442026652447217467319646857167473581888178440868882356750618018595231901049484381823572358726865596222244625618128957441684016103770192396024131410163121117495731019318772306322953690577058407382516406991331784039217070733451951239350429153515009419999393804452602132749715896091569501218683733592930490313741430502701140239342358321299641529399087420936585072698951734762418292915563135699738025426812240568200630240191337712737640468017278273, 17)
Private Key: (n, d) (230562358145369835729651954952002102112471835717574720398687236341927612565177742879617857379737334298856403391302462684500346442584387957288413383634027187878820844978546504420266524472174673196468571674735818881784408688823567506180185952319010494843818235723587268655962222446256181289574416840161037701923960241314101631211174957310

In [24]:
# SENDER

# this cell represents the sender,who only "knows" n and e, doesn't know the private key d

message = input("Enter the message to encrypt: ")


ascii_rep = convert(message)  
ciphertext = encryption(message, n, e)  

print("ASCII (trailing zero if needed):", ascii_rep)
print("Encrypted Message:", ciphertext)

# why ciphertext is a tuple?? the second number in the tuple represents the message length. we need to know the message length in order to figure out if there's a trailing zero in the beginning of the ASCII string. 

Enter the message to encrypt:  Jenson is the smartest person in the whole world!!!


ASCII (trailing zero if needed): 074101110115111110032105115032116104101032115109097114116101115116032112101114115111110032105110032116104101032119104111108101032119111114108100033033033
Encrypted Message: (1518425859873543273677134176840569468894051192191730272637914809642658814917241517685212761300443276269630012679651694585411654912185858474624992230964008224199577617234458672361142777839531985553027751380864216407512545702854580965205527895806654700531166835643992509497330380539326984664260577198081017780905793724418131297796771139383757046393305118291524887439509239793635876351236321183930797845250224114743352270667587236100280931243766935500743166252943969982951220318356234675974465809150997039325942218628665247414023443099312260869019456762608104979980136877718354211403843462989527599148386613930608388846, 153)


In [26]:
# RECEIVER
# this cell represents the receiver, who "knows" the private key d.

import ast

ciphertext_str = input("Enter the encrypted message to decrypt: ")

ciphertext_tuple = ast.literal_eval(ciphertext_str)  # parses string into an actual tuple
decrypted_message = decryption(ciphertext_tuple, n, d)
print("Decrypted Message:", decrypted_message)

Enter the encrypted message to decrypt:  (1518425859873543273677134176840569468894051192191730272637914809642658814917241517685212761300443276269630012679651694585411654912185858474624992230964008224199577617234458672361142777839531985553027751380864216407512545702854580965205527895806654700531166835643992509497330380539326984664260577198081017780905793724418131297796771139383757046393305118291524887439509239793635876351236321183930797845250224114743352270667587236100280931243766935500743166252943969982951220318356234675974465809150997039325942218628665247414023443099312260869019456762608104979980136877718354211403843462989527599148386613930608388846, 153)


Decrypted Message: Jenson is the smartest person in the whole world!!!
